In [1]:
# Importing libraries
import time
import cv2
import numpy as np

In [2]:
confid = 0.5
# The default value for the threshold is 0.5 for normalized predicted probabilities
thresh = 0.5

#Setting up the video path
vid_path = "Test1.mp4" 

In [3]:
def calibrated_dist(p1, p2):
    return ((p1[0] - p2[0]) ** 2 + 550 / ((p1[1] + p2[1]) / 2) * (p1[1] - p2[1]) ** 2) ** 0.5


def isclose(p1, p2):
    c_d = calibrated_dist(p1, p2)
    calib = (p1[1] + p2[1]) / 2
    if 0 < c_d < 0.15 * calib:
        return 1
    elif 0 < c_d < 0.2 * calib:
        return 2
    else:
        return 0

In [4]:
# load the COCO class labels our YOLO model was trained on
labelsPath = "./coco.names"
LABELS = open(labelsPath).read().strip().split("\n")

np.random.seed(42)

# Derive the paths to the YOLO weights and model configuration
weightsPath = "./yolov3.weights"
configPath = "./yolov3.cfg"

###### use this for faster processing (caution: slighly lower accuracy) ###########

# weightsPath = "./yolov3-tiny.weights"  ## https://pjreddie.com/media/files/yolov3-tiny.weights
# configPath = "./yolov3-tiny.cfg"       ## https://github.com/pjreddie/darknet/blob/master/cfg/yolov3-tiny.cfg

In [6]:
# Load our YOLO object detector trained on COCO dataset (80 classes)
net = cv2.dnn.readNetFromDarknet(configPath, weightsPath)

# Determine only the output layer names that we need from YOLO
ln = net.getLayerNames()
ln = [ln[i[0] - 1] for i in net.getUnconnectedOutLayers()]

# Creating object of VideoCapture class and passing vedio name to it.
vs = cv2.VideoCapture(vid_path)
writer = None
(W, H) = (None, None)

fl = 0
q = 0
while True:

    (grabbed, frame) = vs.read()

    if not grabbed:
        break

    if W is None or H is None:
        (H, W) = frame.shape[:2]
        q = W

    frame = frame[0:H, 200:q]
    (H, W) = frame.shape[:2]
    blob = cv2.dnn.blobFromImage(frame, 1 / 255.0, (416, 416),
                                 swapRB=True, crop=False)
    net.setInput(blob)
    start = time.time()
    layerOutputs = net.forward(ln)
    end = time.time()

    boxes = []
    confidences = []
    classIDs = []

    for output in layerOutputs:

        for detection in output:

            scores = detection[5:]
            classID = np.argmax(scores)
            confidence = scores[classID]
            if LABELS[classID] == "person" or LABELS[classID] == "car" or LABELS[classID] == "bus" or LABELS[classID] == "bicycle":

                if confidence > confid: # if prediction is 50%
                    #object detected
                    box = detection[0:4] * np.array([W, H, W, H])
                    (centerX, centerY, width, height) = box.astype("int")
                    
                    # coordinates for rectangle
                    x = int(centerX - (width / 2))
                    y = int(centerY - (height / 2))

                    boxes.append([x, y, int(width), int(height)])
                    confidences.append(float(confidence))
                    classIDs.append(classID)

    idxs = cv2.dnn.NMSBoxes(boxes, confidences, confid, thresh)

    if len(idxs) > 0:

        status = list()
        idf = idxs.flatten()
        close_pair = list()
        s_close_pair = list()
        center = list()
        dist = list()
        
        # Extract the bounding box and centroid coordinates
        for i in idf:
            (x, y) = (boxes[i][0], boxes[i][1])
            (w, h) = (boxes[i][2], boxes[i][3])
            center.append([int(x + w / 2), int(y + h / 2)])

            status.append(0)
        for i in range(len(center)):
            for j in range(len(center)):
                g = isclose(center[i], center[j])

                if g == 1:

                    close_pair.append([center[i], center[j]])
                    status[i] = 1
                    status[j] = 1
                elif g == 2:
                    s_close_pair.append([center[i], center[j]])
                    if status[i] != 1:
                        status[i] = 2
                    if status[j] != 1:
                        status[j] = 2

        total_p = len(center)
        low_risk_p = status.count(2)
        high_risk_p = status.count(1)
        safe_p = status.count(0)
        kk = 0
        
        
        for i in idf:

            sub_img = frame[10:70, 260:W - 260]
            black_rect = np.ones(sub_img.shape, dtype=np.uint8) * 0

            res = cv2.addWeighted(sub_img, 0.77, black_rect, 0.23, 1.0)

            frame[10:70, 260:W - 260] = res

            cv2.putText(frame, "Distance Detection", (380, 50),
                        cv2.FONT_HERSHEY_TRIPLEX, 1, (255, 255, 255), 2)
            

            tot_str = "TOTAL COUNT: " + str(total_p)
            high_str = "IN HIGH RISK ZONE: " + str(high_risk_p)
            low_str = "IN LOW RISK ZONE: " + str(low_risk_p)
            safe_str = "IN SAFE ZONE: " + str(safe_p)

            sub_img = frame[H - 120:H, 0:210]
            black_rect = np.ones(sub_img.shape, dtype=np.uint8) * 0

            res = cv2.addWeighted(sub_img, 0.8, black_rect, 0.2, 1.0)

            cv2.putText(frame, tot_str, (900, H - 90),
                        cv2.FONT_HERSHEY_TRIPLEX, 0.6, (255, 255, 255), 1)
            cv2.putText(frame, safe_str, (895, H - 65),
                        cv2.FONT_HERSHEY_TRIPLEX, 0.6, (0, 255, 0), 1)
            cv2.putText(frame, low_str, (860, H - 40),
                        cv2.FONT_HERSHEY_TRIPLEX, 0.6, (0, 120, 255), 1)
            cv2.putText(frame, high_str, (850, H - 15),
                        cv2.FONT_HERSHEY_TRIPLEX, 0.6, (0, 0, 150), 1)

            (x, y) = (boxes[i][0], boxes[i][1])
            (w, h) = (boxes[i][2], boxes[i][3])
            # If the index pair exists within the violation/abnormal sets, then update the color
            if status[kk] == 1:
                cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 0, 150), 2)

            elif status[kk] == 0:
                cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)

            else:
                cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 120, 255), 2)

            kk += 1
        for h in close_pair:
            cv2.line(frame, tuple(h[0]), tuple(h[1]), (0, 0, 255), 2)
        for b in s_close_pair:
            cv2.line(frame, tuple(b[0]), tuple(b[1]), (0, 255, 255), 2)

        cv2.imshow('Social distancing analyser', frame)
        # key 'e' is used to end the running code.
        if cv2.waitKey(20) & 0xFF == ord('e'):
            break

    if writer is None:
        fourcc = cv2.VideoWriter_fourcc(*"MJPG")  # format of output file
        # Output vedio is written into the output.mp4 file
        writer = cv2.VideoWriter("output.mp4", fourcc, 30,
                                 (frame.shape[1], frame.shape[0]), True)

    writer.write(frame)
print("Processing finished: open output.mp4")
writer.release()
vs.release()
cv2.destroyAllWindows()

Processing finished: open output.mp4
